<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/PIC_Constru%C3%A7%C3%A3o_de_modelos_de_detec%C3%A7%C3%A3o_autom%C3%A1tica_de_linguagem_ofensiva_e_ou_de_%C3%B3dio_em_l%C3%ADngua_portuguesa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas básicas

In [112]:
 !pip install plotly --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
#Importing needed libraries 

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#NLTK tools for text processing 

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

#Modeling packages 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Base de dados Artigo 1 (ToLD-BR)

---



In [ ]:
base_toldbr = pd.read_csv('/content/drive/MyDrive/Dataset/ToLD-BR.csv') #importação da base de dados

In [ ]:
base_toldbr

,text,homophobia,obscene,insult,racism,misogyny,xenophobia
0,Meu nivel de amizade com isis é ela ter meu in...,0.0,0.0,2.0,0.0,0.0,0.0
1,"rt @user @user o cara adultera dados, que fora...",0.0,0.0,1.0,0.0,0.0,0.0
2,@user @user @user o cara só é simplesmente o m...,0.0,2.0,1.0,0.0,0.0,0.0
3,eu to chorando vei vsf e eu nem staneio izone ...,0.0,1.0,0.0,0.0,0.0,0.0
4,Eleitor do Bolsonaro é tão ignorante q não per...,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
20995,@user faz favor vai carai,0.0,0.0,0.0,0.0,0.0,0.0
20996,só queria conhecer alguém que não conhece o he...,1.0,0.0,0.0,0.0,0.0,0.0
20997,"vcs militam na hora errada em cima de memes, p...",0.0,0.0,0.0,0.0,0.0,0.0
20998,@user porra any eu tava c dor de cabeca e fui ...,0.0,0.0,0.0,0.0,0.0,0.0


**Verificando o tamanho do dataset**

In [ ]:
base_toldbr.shape #linhas x colunas

(21000, 7)

**Conferindo o total de textos por categorias**

In [ ]:
base_toldbr['homophobia'].value_counts() 

0.0    20656
1.0      168
2.0      102
3.0       74
Name: homophobia, dtype: int64

In [ ]:
base_toldbr['obscene'].value_counts()

0.0    14348
1.0     4249
2.0     1791
3.0      612
Name: obscene, dtype: int64

In [ ]:
base_toldbr['insult'].value_counts()

0.0    16615
1.0     2516
2.0     1352
3.0      517
Name: insult, dtype: int64

In [ ]:
base_toldbr['racism'].value_counts()

0.0    20862
1.0      105
2.0       27
3.0        6
Name: racism, dtype: int64

In [ ]:
base_toldbr['misogyny'].value_counts()

0.0    20537
1.0      330
2.0      104
3.0       29
Name: misogyny, dtype: int64

In [ ]:
base_toldbr['xenophobia'].value_counts()

0.0    20849
1.0      109
2.0       27
3.0       15
Name: xenophobia, dtype: int64

Verificando valores nulos e tipos de dados

In [ ]:
base_toldbr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        21000 non-null  object 
 1   homophobia  21000 non-null  float64
 2   obscene     21000 non-null  float64
 3   insult      21000 non-null  float64
 4   racism      21000 non-null  float64
 5   misogyny    21000 non-null  float64
 6   xenophobia  21000 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.1+ MB


**Como verificado logo acima não há valores NULL**

In [ ]:
base_toldbr.isnull().sum()

text          0
homophobia    0
obscene       0
insult        0
racism        0
misogyny      0
xenophobia    0
dtype: int64

**Donwload dos pacotes que permitirão fazer os trabalhos de PLN**

**Limpeza dos dados**

**O que é Stop Words?**

Stop words (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi.

In [ ]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_toldbr['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))
  

    


Ranque das palavras únicas com base na frequência de cada palavra


In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


**Rank de palavras com maior frequência sem Stopword** 

In [ ]:
important_unigrams[:5]

,0
co,3619
pra,2999
porra,1941
pqp,1726
caralho,1688


**Rank de palavras com maior frequência com Stopword** 

In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_toldbr['text'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
user,13270
que,8045
de,7814
eu,5644
não,3875


**Verificando a quantidade de palavras na base (com e sem Stopwords)**


In [ ]:
lista_quantidade_palavras_toldbr_com_sw = []
quantidade_palavras_com_sw = 0
for texto in base_toldbr['text']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_toldbr_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_toldbr_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_toldbr_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 325819 palavras
Quantidade de palavras sem Stopwords: 218143 palavras


**Tamanho médio e desvio padrão dos textos em termos do número de palavras para a base como um todo** 

In [ ]:
tam_base = len(base_toldbr['text'])

print(f'Tamanho médio da base com Stop Words: {round(quantidade_palavras_com_sw/tam_base, 2)} ')
print(f'Tamanho médio da base sem Stop Words: {round(quantidade_palavras_sem_sw/tam_base, 2)} \n')

print(f'Desvio padrão da base com Stop Words: {round(np.std(lista_quantidade_palavras_toldbr_com_sw),2)}')
print(f'Desvio padrão da base sem Stop Words: {round(np.std(lista_quantidade_palavras_toldbr_sem_sw),2)}')


Tamanho médio da base com Stop Words: 15.52 
Tamanho médio da base sem Stop Words: 10.39 

Desvio padrão da base com Stop Words: 11.64
Desvio padrão da base sem Stop Words: 7.76


**tamanho médio e desvio padrão dos textos em termos do número de palavras para cada categoria**

homofobia


In [ ]:
homofobia = base_toldbr.loc[base_toldbr['homophobia'] != 0.0 ]

commentarios_homofobicos = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in homofobia['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  commentarios_homofobicos.append(' '.join(filtered_result))

lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in commentarios_homofobicos:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como homofobico: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados homofobico: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como homofobico: 3509 palavras
Tamanho médio da base para comentários rotulados homofobico: 10.2 
Desvio padrão: 7.87


Obceno

In [ ]:
obsceno = base_toldbr.loc[base_toldbr['obscene'] != 0.0 ]
comentarios_obscenos = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in obsceno['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comentarios_obscenos.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_obscenos:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como obsceno: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados obsceno: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')





Quantidade de palavras para texto rotulados como obsceno: 65293 palavras
Tamanho médio da base para comentários rotulados obsceno: 9.82 
Desvio padrão: 7.15


Insulto

In [ ]:
insulto = base_toldbr.loc[base_toldbr['insult'] != 0.0 ]

comentarios_insulto = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in insulto['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comentarios_insulto.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_insulto:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como insulto: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como insulto: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')

Quantidade de palavras para texto rotulados como insulto: 48692 palavras
Tamanho médio da base para comentários rotulados como insulto: 11.1 
Desvio padrão: 8.35


Racismo

In [ ]:
racismo = base_toldbr.loc[base_toldbr['racism'] != 0.0 ]
comentarios_racismo = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in racismo['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comentarios_racismo.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_racismo:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como racismo: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados racismo: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')

Quantidade de palavras para texto rotulados como racismo: 1608 palavras
Tamanho médio da base para comentários rotulados racismo: 11.65 
Desvio padrão: 8.71


Misoginia 

In [ ]:
misoginia = base_toldbr.loc[base_toldbr['misogyny'] != 0.0 ]
comentarios_misoginia = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in misoginia['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comentarios_misoginia.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_obscenos:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como misoginia: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados misoginia: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')

Quantidade de palavras para texto rotulados como misoginia: 65293 palavras
Tamanho médio da base para comentários rotulados misoginia: 9.82 
Desvio padrão: 7.15


Xenofobia 

In [ ]:
xenofobia = base_toldbr.loc[base_toldbr['xenophobia'] != 0.0 ]
comentarios_xenofobia = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in xenofobia['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comentarios_xenofobia.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_xenofobia:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como xenofobia: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados xenofobia: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')



Quantidade de palavras para texto rotulados como xenofobia: 1906 palavras
Tamanho médio da base para comentários rotulados xenofobia: 12.62 
Desvio padrão: 8.96



# Base de dados Artigo 2 (HateBR)

In [ ]:
base_hatebr = pd.read_csv('/content/drive/MyDrive/Dataset/HateBR.csv')

In [ ]:
base_hatebr

,instagram_comments,offensive_language,offensiveness_levels,hate_speech
0,este lixo ...,1,1,-1
1,Mais um lixo,1,1,-1
2,Essa nao tem vergonha na cara!!,1,2,-1
3,Essa mulher é doente.pilantra!,1,3,-1
4,Comunista safada...,1,2,"5,8"
...,...,...,...,...
6995,Time perdendo a credibilidade,0,0,0
6996,Siga em frente Presidente Bolsonaro! Afinal o ...,0,0,0
6997,Tantas coisas importantes para resolver e fica...,0,0,0
6998,"A TIME escolhe quem eles quiser, isso não sign...",0,0,0


**Verificando o tamanho da Base**

In [ ]:
base_hatebr.shape #linhas x colunas

(7000, 4)

**Conferindo o total de textos por categorias**

Com o código abaixo podemos ver que os dados estão balanceados(número de classificação são iguais)

In [ ]:
base_hatebr['offensive_language'].value_counts() 

1    3500
0    3500
Name: offensive_language, dtype: int64

In [ ]:
# np.unique(base_hatebr['offensiveness_levels'], return_counts= True)
base_hatebr['offensiveness_levels'].value_counts() 

0    3500
2    1440
1    1281
3     779
Name: offensiveness_levels, dtype: int64

In [ ]:
# np.unique(base_hatebr['hate_speech'], return_counts= True)
base_hatebr['hate_speech'].value_counts(ascending = True)  #verificar certinho do que se trata estes dados

9         1
5,7       1
3,5       1
4,6       2
1         2
2,5       4
3,8       6
6         6
5,8      11
4        15
3        20
2        28
7        46
8        80
5       479
-1     2798
0      3500
Name: hate_speech, dtype: int64

In [ ]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_hatebr ['instagram_comments']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))
  

In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
presidente,458
est,440
pra,437
brasil,389
ns,380


In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_hatebr ['instagram_comments'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
que,2434
de,2069
não,1264
do,1014
um,782


**Verificando a quantidade de palavras na base (com e sem Stopwords)**


In [ ]:
lista_quantidade_palavras_base_hatebr_com_sw = []
quantidade_palavras_com_sw = 0
for texto in base_hatebr ['instagram_comments']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_hatebr_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_base_hatebr_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_hatebr_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 81007 palavras
Quantidade de palavras sem Stopwords: 53598 palavras


In [ ]:
tam_base = len(base_hatebr ['instagram_comments'])

print(tam_base)

print(f'Tamanho médio da base com Stop Words: {round(quantidade_palavras_com_sw/tam_base, 2)} ')
print(f'Tamanho médio da base sem Stop Words: {round(quantidade_palavras_sem_sw/tam_base, 2)} \n')

print(f'Desvio padrão da base com Stop Words: {round(np.std(lista_quantidade_palavras_base_hatebr_com_sw),2)}')
print(f'Desvio padrão da base sem Stop Words: {round(np.std(lista_quantidade_palavras_base_hatebr_sem_sw),2)}')



7000
Tamanho médio da base com Stop Words: 11.57 
Tamanho médio da base sem Stop Words: 7.66 

Desvio padrão da base com Stop Words: 14.25
Desvio padrão da base sem Stop Words: 9.16


**tamanho médio e desvio padrão dos textos em termos do número de palavras para cada categoria**

Categoria Ofensiva

In [ ]:
offensive_language = base_hatebr.loc[base_hatebr['offensive_language'] == 1 ] #Pegando apenas as informações com linguagem ofensiva 
comentarios_ofensivos = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in offensive_language['instagram_comments']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_ofensivos.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_ofensivos:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como linguagem ofensiva: {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como linguagem ofensivo ofensiva: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como linguagem ofensiva: 28970 palavras
Tamanho médio da base para comentários rotulados como linguagem ofensivo ofensiva: 8.28 
Desvio padrão: 9.26


Categoria não ofensiva

In [ ]:
offensive_language = base_hatebr.loc[base_hatebr['offensive_language'] == 0 ] #Pegando apenas as informações com linguagem não ofensiva 
comentarios_nao_ofensivos = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in offensive_language['instagram_comments']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_nao_ofensivos.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_nao_ofensivos:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)
print(f'Quantidade de palavras para texto rotulados como não ofensivo : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados não ofensivo: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como não ofensivo : 24632 palavras
Tamanho médio da base para comentários rotulados não ofensivo: 7.04 
Desvio padrão: 9.02


# Base de dados Artigo 3 (OffComBR2 e OffComBR3)

In [ ]:
base_offcombr2 = pd.read_csv('/content/drive/MyDrive/Dataset/OffComBR2.csv')

In [ ]:
base_offcombr2

,Class,Texto
0,yes,Votaram no PEZAO Agora tomem no CZAO'
1,no,cuidado com a poupanca pessoal Lembram o que a...
2,no,Sabe o que eu acho engracado os nossos governa...
3,yes,os cariocas tem o que merecem um pessoal que s...
4,no,Podiam retirar dos lucros dos bancos '
...,...,...
1245,yes,LIXO DO CRL'
1246,yes,ela esta fazendo isso para aparecer Nao e gay ...
1247,yes,Volta pra jaula sua macaca vou pegar meu xicot...
1248,yes,Achei que a macaca vivia apenas na floresta ou...


In [ ]:
# np.unique(base_offcombr2['Class'], return_counts= True)
base_offcombr2['Class'].value_counts()

no     831
yes    419
Name: Class, dtype: int64

In [ ]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_offcombr2 ['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))


In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
nao,340
vai,131
so,107
pra,97
ja,75


In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_offcombr2 ['Texto'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
de,474
que,462
nao,340
do,212
da,160


In [ ]:
lista_quantidade_palavras_base_offcombr2_com_sw = []
quantidade_palavras_com_sw = 0
for texto in base_offcombr2['Texto']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_offcombr2_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_base_offcombr2_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_offcombr2_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 17156 palavras
Quantidade de palavras sem Stopwords: 10112 palavras


In [ ]:
tam_base = len(base_offcombr2 ['Texto'])

print(f'Tamanho médio da base com Stop Words: {round(quantidade_palavras_com_sw/tam_base, 2)} ')
print(f'Tamanho médio da base sem Stop Words: {round(quantidade_palavras_sem_sw/tam_base, 2)} \n')

print(f'Desvio padrão da base com Stop Words: {round(np.std(lista_quantidade_palavras_base_offcombr2_com_sw),2)}')
print(f'Desvio padrão da base sem Stop Words: {round(np.std( lista_quantidade_palavras_base_offcombr2_sem_sw),2)}')


Tamanho médio da base com Stop Words: 13.72 
Tamanho médio da base sem Stop Words: 8.09 

Desvio padrão da base com Stop Words: 11.36
Desvio padrão da base sem Stop Words: 6.54


**tamanho médio e desvio padrão dos textos em termos do número de palavras para cada categoria**

Com discurso do ódio

In [ ]:
com_odio = base_offcombr2.loc[base_offcombr2['Class'] == 'yes' ] #Pegando apenas as informações com linguagem não ofensiva 

comentarios_com_odio = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in com_odio['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_com_odio.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_com_odio:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como discurso de ódio : 2789 palavras
Tamanho médio da base para comentários rotulados como discurso de ódio: 6.66 
Desvio padrão: 4.8


Sem Discurso de ódio

In [ ]:
sem_odio = base_offcombr2.loc[base_offcombr2['Class'] == 'no' ] #Pegando apenas as informações com linguagem não ofensiva 

comentarios_sem_odio = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in sem_odio['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_sem_odio.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_sem_odio:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como sem discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como sem  discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como sem discurso de ódio : 7323 palavras
Tamanho médio da base para comentários rotulados como sem  discurso de ódio: 8.81 
Desvio padrão: 7.15


In [ ]:
base_offcombr3 = pd.read_csv('/content/drive/MyDrive/Dataset/OffComBR3.csv')

In [ ]:
base_offcombr3

,Class,Texto
0,yes,Votaram no PEZAO Agora tomem no CZAO'
1,no,cuidado com a poupanca pessoal Lembram o que a...
2,no,Sabe o que eu acho engracado os nossos governa...
3,no,Podiam retirar dos lucros dos bancos '
4,no,CADE O GALVAO PRA NARRAR AGORA FALIIIIUUUUUU...
...,...,...
1028,no,Cruz so tem agilidade mesmo poder de nocaute z...
1029,no,Meus caros amigos enigmaticosNao deveriam comp...
1030,yes,Ele chamava pra atras da escola e sentava nos ...
1031,no,Jhalim Rabei ate fiquei assustado comecei a le...


In [ ]:
# np.unique(base_offcombr2['Class'], return_counts= True)
base_offcombr3['Class'].value_counts()

no     831
yes    202
Name: Class, dtype: int64

In [ ]:
lista_quantidade_palavras_base_offcombr3_com_sw = []
quantidade_palavras_com_sw = 0
for texto in base_offcombr3['Texto']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_offcombr3_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_base_offcombr3_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_offcombr3_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 14632 palavras
Quantidade de palavras sem Stopwords: 10112 palavras


In [ ]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_offcombr3 ['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))


In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
nao,293
vai,99
so,90
pra,81
ja,68


In [ ]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_offcombr3 ['Texto'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
important_unigrams[:5]

,0
que,403
de,399
nao,293
do,177
da,138


In [ ]:
lista_quantidade_palavras_base_offcombr3_com_sw = []
quantidade_palavras_com_sw = 0
for texto in base_offcombr3['Texto']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_offcombr3_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_base_offcombr3_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_offcombr3_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 14632 palavras
Quantidade de palavras sem Stopwords: 8585 palavras


**tamanho médio e desvio padrão dos textos em termos do número de palavras para cada categoria**

Com ódio

In [ ]:
com_odio = base_offcombr3.loc[base_offcombr3['Class'] == 'yes' ] #Pegando apenas as informações com linguagem não ofensiva 

comentarios_com_odio = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in com_odio['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_com_odio.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_com_odio:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como discurso de ódio : 1297 palavras
Tamanho médio da base para comentários rotulados como discurso de ódio: 6.42 
Desvio padrão: 4.9


Sem ódio

In [ ]:
sem_odio = base_offcombr3.loc[base_offcombr3['Class'] == 'no' ] #Pegando apenas as informações com linguagem não ofensiva 

comentarios_sem_odio = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in sem_odio['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_sem_odio.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_sem_odio:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como sem discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como sem discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como sem discurso de ódio : 7288 palavras
Tamanho médio da base para comentários rotulados como sem discurso de ódio: 8.77 
Desvio padrão: 6.96


# Base de dados Artigo 4 (base_hate_speech_binary e hate_speech_hierarchical)

In [114]:
base_hate_speech_binary = pd.read_csv('/content/drive/MyDrive/Dataset/2019-05-28_portuguese_hate_speech_binary_classification.csv')

In [115]:
base_hate_speech_binary

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E
...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A


In [116]:
base_hate_speech_binary['hatespeech_comb'].value_counts() 

0    3882
1    1788
Name: hatespeech_comb, dtype: int64

In [117]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_hate_speech_binary['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))

In [118]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [119]:
important_unigrams[:5]

,0
co,2424
https,2348
rt,1134
mulher,516
pra,491


In [120]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_hate_speech_binary['text'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [121]:
important_unigrams[:5]

,0
de,2457
co,2421
https,2338
que,2141
não,1644


In [122]:
lista_quantidade_palavras_base_hate_speech_binary_com_sw = []
quantidade_palavras_com_sw = 0
for texto in base_hate_speech_binary['text']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_hate_speech_binary_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_base_hate_speech_binary_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_base_hate_speech_binary_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 88619 palavras
Quantidade de palavras sem Stopwords: 71717 palavras


In [123]:
tam_base = len( base_hate_speech_binary['text'])

print(f'Tamanho médio da base com Stop Words: {round(quantidade_palavras_com_sw/tam_base, 2)} ')
print(f'Tamanho médio da base sem Stop Words: {round(quantidade_palavras_sem_sw/tam_base, 2)} \n')

print(f'Desvio padrão da base sem Stop Words: {round(np.std(lista_quantidade_palavras_base_hate_speech_binary_com_sw),2)}')
print(f'Desvio padrão da base com Stop Words: {round(np.std(lista_quantidade_palavras_base_hate_speech_binary_sem_sw),2)}')


Tamanho médio da base com Stop Words: 15.63 
Tamanho médio da base sem Stop Words: 12.65 

Desvio padrão da base sem Stop Words: 6.37
Desvio padrão da base com Stop Words: 4.64


**tamanho médio e desvio padrão dos textos em termos do número de palavras para cada categoria**

Com discurso de ódio

In [124]:
com_odio = base_hate_speech_binary.loc[base_hate_speech_binary['hatespeech_comb'] == 1 ] #Pegando apenas as informações com discurso de ódio 

comentarios_com_odio = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in com_odio['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_com_odio.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_com_odio:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como discurso de ódio : 22335 palavras
Tamanho médio da base para comentários rotulados como discurso de ódio: 12.49 
Desvio padrão: 4.69


Sem discurso de ódio

In [125]:
sem_odio = base_hate_speech_binary.loc[base_hate_speech_binary['hatespeech_comb'] == 0 ] #Pegando apenas as informações com discurso de ódio 

comentarios_sem_odio = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in sem_odio['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_sem_odio.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_sem_odio:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como sem discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como sem discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como sem discurso de ódio : 49382 palavras
Tamanho médio da base para comentários rotulados como sem discurso de ódio: 12.72 
Desvio padrão: 4.61


In [126]:
hate_speech_hierarchical = pd.read_csv('/content/drive/MyDrive/Dataset/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv')

In [127]:
hate_speech_hierarchical

,text,Hate.speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Thin.women,Arabic,East.europeans,Africans,South.Americans,Brazilians,Migrants,Homossexuals,Thin.people,Ageing
0,"""não come mel, morde marimbondo""",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"não tem pinto, tem orgulho !",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Não vê essa merda de Crepúsculo! Pra isso temo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"não da tapa na bundinha, da cotovelada nas cos...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,o diminutivo INHO não acompanha a trajetória d...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,Na minha sala só tem viado e sapatão e a cois...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5664,PARABENS SAPATÃO SDDS @attomiter https://t.co/...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5665,RT @toquedeveludo: Agora um poema:\nEu sou sap...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5666,O mundo das sapatao é mais ligado do que eu im...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [128]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in hate_speech_hierarchical['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))

In [129]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [130]:
important_unigrams[:5]

,0
co,2425
https,2396
rt,1134
mulher,516
pra,491


In [131]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(hate_speech_hierarchical['text'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [132]:
important_unigrams[:5]

,0
de,2457
co,2422
https,2395
que,2140
não,1647


In [133]:
lista_quantidade_palavras_hate_speech_hierarchical_com_sw = []
quantidade_palavras_com_sw = 0
for texto in hate_speech_hierarchical['text']:
  palavras = texto.split(' ')
  lista_quantidade_palavras_hate_speech_hierarchical_com_sw.append(len(palavras))
  quantidade_palavras_com_sw += len(palavras)

lista_quantidade_palavras_hate_speech_hierarchical_sem_sw = []
quantidade_palavras_sem_sw = 0
for texto in comments:
  palavras = texto.split(' ')
  lista_quantidade_palavras_hate_speech_hierarchical_sem_sw.append(len(palavras))
  quantidade_palavras_sem_sw += len(palavras)


print(f'Quantidade de palavras com Stopwords: {quantidade_palavras_com_sw} palavras')
print(f'Quantidade de palavras sem Stopwords: {quantidade_palavras_sem_sw} palavras')

Quantidade de palavras com Stopwords: 88777 palavras
Quantidade de palavras sem Stopwords: 71762 palavras


In [134]:
tam_base = len(hate_speech_hierarchical['text'])

print(f'Tamanho médio da base com Stop Words: {round(quantidade_palavras_com_sw/tam_base, 2)} ')
print(f'Tamanho médio da base sem Stop Words: {round(quantidade_palavras_sem_sw/tam_base, 2)} \n')

print(f'Desvio padrão da base sem Stop Words: {round(np.std(lista_quantidade_palavras_hate_speech_hierarchical_com_sw),2)}')
print(f'Desvio padrão da base com Stop Words: {round(np.std(lista_quantidade_palavras_hate_speech_hierarchical_sem_sw),2)}')


Tamanho médio da base com Stop Words: 15.66 
Tamanho médio da base sem Stop Words: 12.66 

Desvio padrão da base sem Stop Words: 6.36
Desvio padrão da base com Stop Words: 4.64


**Tamanho médio e desvio padrão dos textos em termos do número de palavras para cada categoria**

**Como neste dataset há um grande número de categorias foi realizado o agrupamento por categoria, ficando da seguinte maneira:**

**Hate speech** - Sexism, Homophobia, Body, Racism, Ideology,  Religion, Origin, OtherLifestyle, Migrants, Health,  Ageing.

**Origin** - Latins, Rural people, Asians, South Americans, Africans, Arabic,  East europeans, Brazilians,  Brazilian women, Angolans, Nordestines,  Sertanejos, Chinese, Japaneses, Argentines, Mexicans,  Venezuelans, Egyptians, Iranians, Russians, Ucranians.

**Migrants**: Refugees, Immigrants.

**Homophobia** - Homossexuals,  Bissexuals, Lesbians, Gays.

**Sexism** - Women, Men, Feminists, Transexuals,  Men Feminists, Trans women, Travestis, Aborting women, Blond women, Thin women,  Brazilian women,  Football players women,  Homeless women, Muslim women, Old women, Lesbians, Fat women, Ugly women, Black Women

**Body** - Fat people, Ugly people,  Blond women,  Thin people, Fat women, Ugly women.

**Racism** - Black people, Latins, Asians,  White people, Indigenous, Black Women.

**Ideology** - Feminists, Left wing ideology, Agnostic,  Men Feminists.

**Religion** - Islamists, Muslims,  Jews,  Muslim women.

**OtherLifestyle** - Criminals,  Homeless,  Jornalists,  Vegetarians, Gamers,  Polyamorous,  Poor people,  Street artists, Football players women, Homeless women.

**Health** - Disabled people, Autists.

**Ageing** -  Old people,  Young people, Old women, 



Cálculo para comentários que possuem discurso de ódio.

In [135]:
Hate_speech = hate_speech_hierarchical.loc[hate_speech_hierarchical['Hate.speech'] == 1 ]

comentarios_Hate_speech = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in Hate_speech['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_Hate_speech.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_Hate_speech:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como discurso de ódio : 14384 palavras
Tamanho médio da base para comentários rotulados como discurso de ódio: 11.71 
Desvio padrão: 4.65


Cálculo para comentários que não possuem discurso de ódio.

In [163]:
No_Hate_speech = hate_speech_hierarchical.loc[hate_speech_hierarchical['Hate.speech'] == 0 ]

comentarios_No_Hate_speech = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in No_Hate_speech['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comentarios_No_Hate_speech.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in comentarios_No_Hate_speech:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como sem  discurso de ódio : {quantidade_palavras} palavras')
print(f'Tamanho médio da base para comentários rotulados como sem discurso de ódio: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como sem  discurso de ódio : 57378 palavras
Tamanho médio da base para comentários rotulados como sem discurso de ódio: 12.92 
Desvio padrão: 4.61


**Origin**

Origin - Latins, Rural people, Asians, South Americans, Africans, Arabic, East europeans, Brazilians, Brazilian women, Angolans, Nordestines, Sertanejos, Chinese, Japaneses, Argentines, Mexicans, Venezuelans, Egyptians, Iranians, Russians, Ucranians.

In [138]:
latins = hate_speech_hierarchical.loc[hate_speech_hierarchical['Latins'] == 1 ]
rural_people = hate_speech_hierarchical.loc[hate_speech_hierarchical['Rural.people'] == 1 ]
asians = hate_speech_hierarchical.loc[hate_speech_hierarchical['Asians'] == 1 ]
south_america = hate_speech_hierarchical.loc[hate_speech_hierarchical['South.Americans'] == 1 ]
africans = hate_speech_hierarchical.loc[hate_speech_hierarchical['Africans'] == 1 ]
arabic = hate_speech_hierarchical.loc[hate_speech_hierarchical['Arabic'] == 1 ]
east_europeans = hate_speech_hierarchical.loc[hate_speech_hierarchical['East.europeans'] == 1 ]
brazilians = hate_speech_hierarchical.loc[hate_speech_hierarchical['Brazilians'] == 1 ]
brazilian_women = hate_speech_hierarchical.loc[hate_speech_hierarchical['Brazilians.women'] == 1 ]
angolans = hate_speech_hierarchical.loc[hate_speech_hierarchical['Angolans'] == 1 ]
nordestines = hate_speech_hierarchical.loc[hate_speech_hierarchical['Nordestines'] == 1 ]
sertanejos = hate_speech_hierarchical.loc[hate_speech_hierarchical['Sertanejos'] == 1 ]
chinese = hate_speech_hierarchical.loc[hate_speech_hierarchical['Chinese'] == 1 ]
japoneses = hate_speech_hierarchical.loc[hate_speech_hierarchical['Japaneses'] == 1 ]
argentines = hate_speech_hierarchical.loc[hate_speech_hierarchical['Argentines'] == 1 ]
mexicans = hate_speech_hierarchical.loc[hate_speech_hierarchical['Mexicans'] == 1 ]
venezuelans = hate_speech_hierarchical.loc[hate_speech_hierarchical['Venezuelans'] == 1 ]
egyptians = hate_speech_hierarchical.loc[hate_speech_hierarchical['Egyptians'] == 1 ]
iranians = hate_speech_hierarchical.loc[hate_speech_hierarchical['Iranians'] == 1 ]
russians = hate_speech_hierarchical.loc[hate_speech_hierarchical['Russians'] == 1 ]
ucranians = hate_speech_hierarchical.loc[hate_speech_hierarchical['Ucranians'] == 1 ]


Origin = [latins, rural_people, asians, south_america, africans, arabic, east_europeans, brazilians, brazilian_women, angolans, nordestines, sertanejos, chinese, japoneses, argentines, mexicans, venezuelans, egyptians, iranians, russians, ucranians  ]
origin = pd.concat(Origin)



In [164]:

categoria_origin = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in origin['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  categoria_origin.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_origin:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com origem: {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como categoria Origin : 757 palavras
Tamanho médio da base para comentários: 13.28 
Desvio padrão: 4.46


**Migrants**: Refugees, Immigrants

In [162]:
refugees = hate_speech_hierarchical.loc[hate_speech_hierarchical['Refugees'] == 1 ]
immigrants = hate_speech_hierarchical.loc[hate_speech_hierarchical['Immigrants'] == 1 ]

migrants = pd.concat([refugees, immigrants])

In [141]:

categoria_migrants = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in migrants['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_migrants.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_migrants:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com migrantes: {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 1239 palavras
Tamanho médio da base: 14.58 
Desvio padrão: 3.11


**Homophobia** - Homossexuals, Bissexuals, Lesbians, Gays.

In [142]:
homophobia = hate_speech_hierarchical.loc[hate_speech_hierarchical['Homophobia'] == 1 ]


In [143]:
categoria_homophobia = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in homophobia['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_homophobia.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_homophobia:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com homophobia : {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 3207 palavras
Tamanho médio da base: 9.96 
Desvio padrão: 4.39


**Sexism** - Women, Men, Feminists, Transexuals, Men Feminists, Trans women, Travestis, Aborting women, Blond women, Thin women, Brazilian women, Football players women, Homeless women, Muslim women, Old women, Lesbians, Fat women, Ugly women, Black Women

In [161]:
sexism = hate_speech_hierarchical.loc[hate_speech_hierarchical['Sexism'] == 1 ]


In [145]:
categoria_sexism = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in sexism['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_sexism.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_sexism:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com sexismo: {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 8161 palavras
Tamanho médio da base: 12.14 
Desvio padrão: 4.58


**Body** - Fat people, Ugly people, Blond women, Thin people, Fat women, Ugly women.

In [146]:
body = hate_speech_hierarchical.loc[hate_speech_hierarchical['Body'] == 1 ]

In [147]:
categoria_body = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in body['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_body.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_body:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com corpo: {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 2061 palavras
Tamanho médio da base: 12.57 
Desvio padrão: 4.74


**Racism** - Black people, Latins, Asians, White people, Indigenous, Black Women.

In [148]:
racism = hate_speech_hierarchical.loc[hate_speech_hierarchical['Racism'] == 1 ]

In [149]:
categoria_racism = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in racism['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_racism.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_racism:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com racismo: {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 1022 palavras
Tamanho médio da base: 10.87 
Desvio padrão: 4.92


**Ideology** - Feminists, Left wing ideology, Agnostic, Men Feminists.

In [160]:
ideology = hate_speech_hierarchical.loc[hate_speech_hierarchical['Ideology'] == 1 ]


In [151]:
categoria_ideology = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in ideology['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_ideology.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_ideology:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com ideologia : {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 1220 palavras
Tamanho médio da base: 13.26 
Desvio padrão: 4.47


**Religion** - Islamists, Muslims, Jews, Muslim women.

In [152]:
religion = hate_speech_hierarchical.loc[hate_speech_hierarchical['Religion'] == 1 ]

In [153]:
categoria_religion = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in religion['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_religion.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_religion:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com religião : {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 427 palavras
Tamanho médio da base: 14.23 
Desvio padrão: 4.29


**OtherLifestyle** - Criminals, Homeless, Jornalists, Vegetarians, Gamers, Polyamorous, Poor people, Street artists, Football players women, Homeless women.

In [154]:
otherlifestyle = hate_speech_hierarchical.loc[hate_speech_hierarchical['OtherLifestyle'] == 1 ]

In [155]:
categoria_otherslifestyle = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in otherlifestyle['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_otherslifestyle.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_otherslifestyle:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com Outros estilos de vida : {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 249 palavras
Tamanho médio da base: 12.45 
Desvio padrão: 4.36


**Health** - Disabled people, Autists.

In [156]:
health = hate_speech_hierarchical.loc[hate_speech_hierarchical['Health'] == 1 ]

In [157]:
categoria_health = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in health['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_health.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_health:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como discurso de ódio com saúde : {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 72 palavras
Tamanho médio da base: 12.0 
Desvio padrão: 2.58


**Ageing** - Old people, Young people, Old women,

In [158]:
ageing = hate_speech_hierarchical.loc[hate_speech_hierarchical['Ageing'] == 1 ]

In [159]:
categoria_ageing = [] 
stop_words = set(stopwords.words('portuguese')) 

for words in ageing['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words)
  tokens = nltk.word_tokenize(only_latters) 
  lower_case = [l.lower() for l in tokens] 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) 
  categoria_ageing.append(' '.join(filtered_result))


lista_quantidade_palavras = []
quantidade_palavras = 0
for texto in categoria_ageing:
  palavras = texto.split(' ')
  lista_quantidade_palavras.append(len(palavras))
  quantidade_palavras += len(palavras)

print(f'Quantidade de palavras para texto rotulados como envelhecimento : {quantidade_palavras} palavras')
print(f'Tamanho médio da base: {round(quantidade_palavras/len(lista_quantidade_palavras), 2)} ')
print(f'Desvio padrão: {round(np.std(lista_quantidade_palavras),2)}')


Quantidade de palavras para texto rotulados como migrants : 55 palavras
Tamanho médio da base: 13.75 
Desvio padrão: 3.42
